In [12]:
from dotenv import dotenv_values

In [21]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY']=openapi_key

In [14]:
!pip install langchain
!pip install langchain langchain_community
!pip install langchain_openai

In [19]:
from langchain.llms import OpenAI

In [22]:
llm = OpenAI(temperature=0.6)

In [27]:
name = llm('I want to open a restaurant for Italian food. Suggest a fancy name for this')
print(name)



"La Bella Cucina" 


In [30]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
    )
prompt_template_name.format(cuisine='Mexican')

'I want to open a restaurant for Mexican food. Suggest a fancy name for this'

In [36]:
from langchain.chains import LLMChain
name_chain = LLMChain(llm=llm,
                prompt=prompt_template_name)
name_chain.invoke('Mexican')

{'cuisine': 'Mexican', 'text': '\n\n"Casa de Sabores"'}

In [42]:
prompt_food_name = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some food menu items for {restaurant_name} restaurant. Return it as comma seperated list"
    )
# prompt_food_name.format(restaurant_name='')
food_items_chain = LLMChain(llm=llm,
                prompt=prompt_food_name)
print(food_items_chain.invoke('Casa de Sabores'))

{'restaurant_name': 'Casa de Sabores', 'text': '\n\n1. Enchiladas de Pollo\n2. Carne Asada Tacos\n3. Arroz con Camarones\n4. Quesadillas de Queso\n5. Fajitas de Res\n6. Sopa de Tortilla\n7. Chile Relleno\n8. Tostadas de Ceviche\n9. Tamales de Elote\n10. Flautas de Pollo\n11. Burrito de Carne Molida\n12. Empanadas de Queso\n13. Pescado a la Veracruzana\n14. Huevos Rancheros\n15. Chimichangas de Pollo\n16. Ceviche de Camarón\n17. Pozole\n18. Chiles en Nogada\n19. Chilaquiles con Pollo\n20. Tacos al Pastor.'}


In [46]:
# Simple sequential chain just take 1 input and give 1 output. Restaurant name is not given in output along with food items menu
from langchain.chains import SimpleSequentialChain
llm = OpenAI(temperature=0.7)
prompt_restaurant_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
    )
name_chain = LLMChain(llm=llm,prompt=prompt_restaurant_name)
prompt_food_name = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some food vegan menu items for {restaurant_name} restaurant. Return it as comma seperated list"
    )
food_items_chain=LLMChain(llm=llm,prompt=prompt_food_name)
sschain = SimpleSequentialChain(chains=[name_chain,food_items_chain])
response = sschain.run('Indian')
print(response)



1. Vegetable Samosas
2. Chana Masala
3. Aloo Gobi
4. Palak Paneer (made with tofu)
5. Dal Tadka
6. Vegetable Biryani
7. Tandoori Vegetables
8. Baingan Bharta
9. Malai Kofta (made with vegan meatballs)
10. Mushroom Masala
11. Aloo Tikki Chaat
12. Dal Makhani
13. Bhindi Masala
14. Vegetable Korma
15. Gobi Manchurian
16. Paneer Tikka (made with tofu)
17. Aloo Palak
18. Mixed Vegetable Curry
19. Tofu Tikka Masala
20. Vegetable Jalfrezi


In [65]:
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
llm=OpenAI(temperature=0.7)
prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open {cuisine} restaurant. Suggest me a fancy name. one name please.")
restaurant_chain = LLMChain(llm=llm,prompt=prompt_template_name,output_key='restaurant_name')
prompt_template_name=PromptTemplate(
    input_variables=['restaurant_name'],
    template='Suggest food vegan menu items for {restaurant_name} restaurant. Return comma seperated list.'
)
food_items_chain = LLMChain(llm=llm,prompt=prompt_template_name,output_key='food_items_menu')
schain = SequentialChain(
    chains=[restaurant_chain,food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name','food_items_menu'])
response = schain.invoke({'cuisine':'Indian'})
print(response)

{'cuisine': 'Indian', 'restaurant_name': '\n\n"Spice Symphony"', 'food_items_menu': '\n\n1. Cauliflower "wings" with buffalo or BBQ sauce\n2. Tofu tikka masala\n3. Vegetable biryani\n4. Vegan tikka masala pizza\n5. Jackfruit "pulled pork" sliders\n6. Lentil and vegetable curry\n7. Vegan sushi rolls (avocado, cucumber, and carrot)\n8. Grilled portobello mushroom burger\n9. Coconut curry vegetables\n10. Vegan pad thai with tofu\n11. Spicy tofu lettuce wraps\n12. Vegan kebabs with grilled vegetables and tofu\n13. Mushroom and spinach tikka masala\n14. Vegetable samosas\n15. Vegan mango lassi smoothie.'}
